In [1]:
import numpy as np 
import pandas as pd 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [6]:
data = pd.read_csv('dataset/train.tsv',sep='\t')

In [7]:
data = data.reindex(np.random.permutation(data.index))
labels = to_categorical(data['Sentiment'], num_classes=5)

In [8]:
epochs = 10
emb_dim = 128
batch_size = 32
labels[:3]

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

### Do the tokenizer and pad_sequences before put into model

In [10]:
n_most_common_words = 8000
tokenizer = Tokenizer(num_words=n_most_common_words)
tokenizer.fit_on_texts(data['Phrase'].values)
sequences = tokenizer.texts_to_sequences(data['Phrase'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences)

Found 15288 unique tokens.


#### split train test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.2, random_state=42)

In [12]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X2.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

W0809 23:34:02.867243 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 23:34:02.944011 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 23:34:02.950213 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 23:34:03.016197 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0809 23:34:03.062700 4545234368 deprecati

((124848, 48), (124848, 5), (31212, 48), (31212, 5))


W0809 23:34:03.908046 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 23:34:03.955439 4545234368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 128)           1024000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 1,156,229
Trainable params: 1,156,229
Non-trainable params: 0
_________________________________________________________________
None


W0809 23:34:04.343931 4545234368 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 99878 samples, validate on 24970 samples
Epoch 1/10
99878/99878 [==============================] - 390s 4ms/step - loss: 1.0073 - acc: 0.5920 - val_loss: 0.8978 - val_acc: 0.6272
Epoch 2/10
99878/99878 [==============================] - 279s 3ms/step - loss: 0.8416 - acc: 0.6536 - val_loss: 0.8559 - val_acc: 0.6526
Epoch 3/10
99878/99878 [==============================] - 252s 3ms/step - loss: 0.7813 - acc: 0.6811 - val_loss: 0.8488 - val_acc: 0.6545
Epoch 4/10
99878/99878 [==============================] - 263s 3ms/step - loss: 0.7376 - acc: 0.6986 - val_loss: 0.8490 - val_acc: 0.6604
Epoch 5/10
99878/99878 [==============================] - 261s 3ms/step - loss: 0.7023 - acc: 0.7150 - val_loss: 0.8530 - val_acc: 0.6600
Epoch 6/10
99878/99878 [==============================] - 262s 3ms/step - loss: 0.6768 - acc: 0.7240 - val_loss: 0.8618 - val_acc: 0.6601
Epoch 7/10
99878/99878 [==============================] - 267s 3ms/step - loss: 0.6524 - acc: 0.7332 - val_loss: 0.8801 - 

## accuracy of the splitted test

In [13]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

31212/31212 [==============================] - 8s 259us/step
Test set
  Loss: 0.938
  Accuracy: 0.652
